# Labeled Faces In The Wild classification


## Desription

The LFW (Labeled Faces In The Wild) dataset is composed of 5479 clases (different persons) for more than 13000 images.

In [18]:
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=5, resize=1.0)

n_samples, h, w = lfw_people.images.shape
n_features = lfw_people.data.shape[1]

X = lfw_people.images
y = lfw_people.target

target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)
print(f"Image dimensions {w}x{h}x1")

print(target_names)


Total dataset size:
n_samples: 5985
n_features: 11750
n_classes: 423
Image dimensions 94x125x1
['Abdullah Gul' 'Adrien Brody' 'Ahmed Chalabi' 'Ai Sugiyama' 'Al Gore'
 'Al Sharpton' 'Alan Greenspan' 'Alastair Campbell' 'Albert Costa'
 'Alejandro Toledo' 'Ali Naimi' 'Allyson Felix' 'Alvaro Uribe'
 'Amelia Vega' 'Amelie Mauresmo' 'Ana Guevara' 'Ana Palacio'
 'Andre Agassi' 'Andy Roddick' 'Angela Bassett' 'Angela Merkel'
 'Angelina Jolie' 'Ann Veneman' 'Anna Kournikova' 'Antonio Banderas'
 'Antonio Palocci' 'Ari Fleischer' 'Ariel Sharon' 'Arminio Fraga'
 'Arnold Schwarzenegger' 'Arnoldo Aleman' 'Ashanti' 'Atal Bihari Vajpayee'
 'Ben Affleck' 'Benazir Bhutto' 'Benjamin Netanyahu' 'Bernard Law'
 'Bertie Ahern' 'Bill Clinton' 'Bill Frist' 'Bill Gates' 'Bill Graham'
 'Bill McBride' 'Bill Simon' 'Billy Crystal' 'Binyamin Ben-Eliezer'
 'Bob Graham' 'Bob Hope' 'Bob Stoops' 'Boris Becker' 'Brad Johnson'
 'Britney Spears' 'Bulent Ecevit' 'Calista Flockhart' 'Cameron Diaz'
 'Carla Del Ponte' 'Carlos

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [20]:
# from sklearn.decomposition import PCA
#
# n_components = 150
# pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
# eigenfaces = pca.components_.reshape((n_components, h, w))
# X_train_pca = pca.transform(X_train)
# X_test_pca = pca.transform(X_test)


In [21]:
from tensorflow import keras
from keras.applications import ResNet50

model = ResNet50
preprocess_input = keras.applications.resnet.preprocess_input


In [22]:
# from keras.preprocessing.image import ImageDataGenerator
#
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.05,
#     zoom_range=0.1)


In [25]:
from keras.optimizers import SGD
from keras import Model
from keras.layers import Flatten, Dense

model = ResNet50(
    input_shape=[250, 250, 3],
    weights="imagenet",
    include_top=False,
    classes=n_samples,
)

# Freeze existing VGG already trained weights
for layer in model.layers[: 10]:
    layer.trainable = False

# get the VGG output
out = model.output

# Add new dense layer at the end
x = Flatten()(out)
x = Dense(n_samples, activation="softmax")(x)

model = Model(inputs=model.input, outputs=x)

opti = SGD(
    lr=0.001,
    momentum=0.9,
    nesterov=True,
)

model.compile(loss="categorical_crossentropy", optimizer=opti, metrics=["accuracy"])

model.summary()


ResourceExhaustedError: OOM when allocating tensor with shape[131072,5985] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model.fit(X_train, y_train, X_test, y_test)


In [ ]:
model.save('weights.h5')
